# Openai格式代码调用优化

- 可调整参数：
    - `max_tokens`：用于指定生成的文本的最大长度。一个`token`可以是一个字、一个词或者一个标点符号。
    - `temperature`：用于控制生成文本的随机性。范围为0-2，值越大，生成的文本越随机，越有创造性；值越小，生成的文本越确定，越准确。
    - `n`：用于指定生成多少个独立的文本片段，也就是多个备选答案。


我们根据这些参数来调整[刚刚的代码](./01_base.ipynb)：

In [1]:
from openai import OpenAI
OPENAI_BASE_URL = 'http://localhost:13000/v1'
client = OpenAI(
     api_key='sk-6n3IuwDfeMEZggn2A61a818dC722474292D929C27004Fe82',
     base_url=OPENAI_BASE_URL
)

In [2]:
response = client.chat.completions.create(
    model="qwen2:7b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
    max_tokens=20,
    temperature=0,
    n=2,
)

In [3]:
response

ChatCompletion(id='chatcmpl-7fb22cb2c14c4fbdb225ed2d4291cd44', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was won by the Los Angeles Dodgers. They defeated the Tampa', role='assistant', function_call=None, tool_calls=None))], created=1731999739, model='qwen2:7b', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=31, total_tokens=51))

In [4]:
response.choices[0].message.content

'The 2020 World Series was won by the Los Angeles Dodgers. They defeated the Tampa'

可以看到减小max_tokens参数之后，回答明显被截断了。

这里n=2，choices还可以被迭代遍历：

In [5]:
for item in response.choices:
    print(item.message.content)

The 2020 World Series was won by the Los Angeles Dodgers. They defeated the Tampa


一般情况下，n取1即可，我们不需要一个问题生成两个回答。

## 特殊的可调整参数：流式输出（逐字生成）

In [10]:
response = client.chat.completions.create(
    model="qwen2:7b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
    stream=True
)

response

可以看到流式输出下，response变成了一个Object，我们来遍历看看：

In [7]:

for item in response:
    print(item)

ChatCompletionChunk(id='chatcmpl-c8bba29de4db410692bb4f7bf9b5dc4f', choices=[Choice(delta=ChoiceDelta(content='The', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1731999739, model='qwen2:7b', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-7391a65581b74ba79276ff94daabd073', choices=[Choice(delta=ChoiceDelta(content=' ', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1731999739, model='qwen2:7b', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-604a99758a234f9787222bcbb43568c8', choices=[Choice(delta=ChoiceDelta(content='2', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1731999739, model='qwen2:7b', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='ch

流式输出就是一个字一个字蹦出来。

查看完整内容：

In [11]:
for item in response:
    content = item.choices[0].delta.content
    print(content, end='')

The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays in six games to claim their seventh championship title in franchise history.

## `.env`文件的使用

一般而言，工程中的代码是不会出现llm调用的url和api-key的，我们将其放在同目录下的`.env`文件：

```
OPENAI_BASE_URL = 'http://localhost:13000/v1'
OPENAI_API_KEY = 'sk-6n3IuwDfeMEZggn2A61a818dC722474292D929C27004Fe82'
```

In [9]:
%pip install python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
client = OpenAI()

response = client.chat.completions.create(
    model="qwen2:7b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
    stream=True
)

for item in response:
    content = item.choices[0].delta.content
    if content is not None:
        print(content, end='')

The 2020 World Series was won by the Los Angeles Dodgers. They defeated the Tampa Bay Rays in six games. It was the Dodgers' first World Series victory since 1988.